In [1]:
import pandas as pd
parent_folder = '/Users/username/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/username/Desktop/ML/Recommendations/arcane/')

import numpy as np
import pandas as pd
import json
from sql.articles.MongoDBArticle import MongoDBArticle
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [2]:
data_df = pd.read_csv('titles_classes_gpt4_results.csv')

In [3]:
data_article_ids = list(data_df.id.unique())

In [4]:
data_articles = MongoDBArticle.fetch_documents_by_ids(data_article_ids)

In [5]:
data_articles_dict = {str(x['_id']): x for x in data_articles}

In [6]:
for idx, x in data_articles_dict.items():
    x['full_content'] = f"title: {x['title']}. content: {x['cleaned_text']}"

In [8]:
data_df['full_content'] = data_df['id'].apply(lambda x: data_articles_dict[x]['full_content'])

In [10]:
data_df.to_csv('titles_classes_gpt4_with_full_content.csv',index = False)

In [11]:
import boto3
iam = boto3.client('iam')
response = iam.list_roles()
sagemaker_roles = [role for role in response['Roles'] if 'SageMaker' in role['RoleName']]
sagemaker_roles[0]['RoleName']

'AmazonSageMaker-ExecutionRole-20231030T210397'

In [12]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231030T210397')['Role']['Arn']
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/username/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/username/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/username/Library/Application Support/sagemaker/config.yaml


Couldn't call 'get_role' to get Role ARN from role name username to get Role path.


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/username/Library/Application Support/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


In [14]:
from io import StringIO # python3; python2: BytesIO 

In [19]:
csv_buffer = StringIO()
data_df.to_csv(csv_buffer)

In [20]:
bucket = sess.default_bucket() # already created on S3
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'datasets/recommendations/title_category_training_data.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'WVMZXG5ST8QJQ2W4',
  'HostId': 'JlD0qOlmALnGb4LSbfjbFE7bzsZJCO2nMNiTCry0ampRH7DiLW7QKhkiiOsQhN5odXGUplcxfeQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'JlD0qOlmALnGb4LSbfjbFE7bzsZJCO2nMNiTCry0ampRH7DiLW7QKhkiiOsQhN5odXGUplcxfeQ=',
   'x-amz-request-id': 'WVMZXG5ST8QJQ2W4',
   'date': 'Sat, 17 Feb 2024 06:30:50 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"f89655f8a08edb25c45983d19da8d646"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"f89655f8a08edb25c45983d19da8d646"',
 'ServerSideEncryption': 'AES256'}

In [ ]:
training_input_path = f's3://{sess.default_bucket()}/fine_tuning_datasets/{finetune_dataset_config["date"]}-{finetune_dataset_config["finetune_id"]}'
lm_dataset.save_to_disk(training_input_path)